In [1]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
4073,application_1713270977862_4404,pyspark,idle,Link,Link,None,
4074,application_1713270977862_4405,pyspark,idle,Link,Link,None,
4084,application_1713270977862_4413,pyspark,starting,Link,Link,None,


In [2]:
print(f'Start Spark name:{spark._sc.appName}, version:{spark.version}')

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
4085,application_1713270977862_4414,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Start Spark name:livy-session-4085, version:3.3.2.3.3.7190.2-1

In [3]:
%%local
import os
default_db = 'com490'
username=os.getenv('USER', 'anonymous')
hadoop_fs=os.getenv('HADOOP_DEFAULT_FS', 'hdfs://iccluster067.iccluster.epfl.ch:8020')
print(f"local username={username}\nhadoop_fs={hadoop_fs}")

local username=kvaerum
hadoop_fs=hdfs://iccluster067.iccluster.epfl.ch:8020


### Data Retrieval

In [4]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

# List files in the directory
files = sc._jvm.org.apache.hadoop.fs.FileSystem.get(sc._jsc.hadoopConfiguration()) \
    .listStatus(sc._jvm.org.apache.hadoop.fs.Path('/data/geo/json'))
for file_status in files:
    print(file_status.getPath())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

hdfs://iccluster067.iccluster.epfl.ch:8020/data/geo/json/country=CH

In [37]:
geo_df = spark.read.json('/data/geo/json')
geo_df = geo_df.filter((geo_df.properties.CANTON == "VD") & (geo_df.properties.NAME == "Lausanne"))
lausanne_geometry = geo_df.select("geometry.coordinates").collect()[0][0]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Istdaten Processing

### 1) Select Relevant Columns and Rename for Convenience

In [42]:
from pyspark.sql.types import DoubleType
istdaten_df = spark.read.orc('/data/sbb/orc/istdaten/year=2023')

istdaten_df = istdaten_df.select([
      istdaten_df['betriebstag'].alias('date'),
      istdaten_df['fahrt_bezeichner'].alias('trip_id'),
      istdaten_df['betreiber_id'].alias('operator_id'),
      istdaten_df['bpuic'].alias('stop_id'),
      istdaten_df['produkt_id'].alias('transportation'),
      istdaten_df['faellt_aus_tf'].alias('cancelled'),
      istdaten_df['ankunftszeit'].alias('arrival_time'),
      istdaten_df['an_prognose'].alias('arrival_prognosis'),
      istdaten_df['an_prognose_status'].alias('arr_prog_status'),
      istdaten_df['abfahrtszeit'].alias('departure_time'),
      istdaten_df['ab_prognose'].alias('departure_prognosis'),
      istdaten_df['ab_prognose_status'].alias('dep_prog_status'),
])

stops_df = spark.read.csv('/data/sbb/csv/timetables/stops', header=True)\
                .select(['stop_id','stop_lat', 'stop_lon'])\
                .drop(*['year','month','day'])\
                .dropDuplicates(['stop_id'])\
                .withColumnRenamed("stop_id", "stops_stop_id")\
                .withColumn("stop_lat", col("stop_lat").cast(DoubleType()))\
                .withColumn("stop_lon", col("stop_lon").cast(DoubleType()))

istdaten_df = istdaten_df.join(
    stops_df,
    istdaten_df.stop_id == stops_df.stops_stop_id,
    "left"
).drop('stops_stop_id')

istdaten_df.show(5)
istdaten_df.printSchema()
istdaten_df.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+--------------------+-----------+-------+--------------+---------+----------------+-------------------+---------------+----------------+-------------------+---------------+----------------+----------------+
|      date|             trip_id|operator_id|stop_id|transportation|cancelled|    arrival_time|  arrival_prognosis|arr_prog_status|  departure_time|departure_prognosis|dep_prog_status|        stop_lat|        stop_lon|
+----------+--------------------+-----------+-------+--------------+---------+----------------+-------------------+---------------+----------------+-------------------+---------------+----------------+----------------+
|26.09.2023|      85:11:8958:001|      85:11|8502128|           Zug|    false|26.09.2023 15:44|26.09.2023 15:45:41|           REAL|26.09.2023 15:44|26.09.2023 15:46:28|           REAL|47.4286409197269|8.16662018167874|
|17.04.2023|      85:11:8925:001|      85:11|8502128|           Zug|    false|17.04.2023 07:08|17.04.2023 07:08:46|         

In [67]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.sql.types import BooleanType

# Parse the strings and remove the altitude
lausanne_polygon = []
for coord_str in lausanne_geometry[0][0]:
    coord = json.loads(coord_str.replace('\'', '\"'))
    lausanne_polygon.append((coord[0], coord[1]))

# Broadcast the polygon to all nodes
broadcast_polygon = spark.sparkContext.broadcast(lausanne_polygon)

# Define a UDF to check if a point is within the Lausanne polygon
def is_within_lausanne(lat, lon):
    point = (lon, lat)  # Note: Point takes (lon, lat)
    polygon_path = Path(broadcast_polygon.value)
    return polygon_path.contains_point(point)

# Register the UDF
is_within_lausanne_udf = udf(is_within_lausanne, BooleanType())

istdaten_df = istdaten_df.filter(is_within_lausanne_udf(istdaten_df.stop_lat, istdaten_df.stop_lon))

istdaten_df.show(5)
istdaten_df.printSchema()
istdaten_df.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:

  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "<stdin>", line 15, in is_within_lausanne
  File "/usr/local/lib/python3.8/dist-packages/matplotlib/path.py", line 129, in __init__
    vertices = _to_unmasked_float_array(vertices)
  File "/usr/local/lib/python3.8/dist-packages/matplotlib/cbook/__init__.py", line 1345, in _to_unmasked_float_array
    return np.asarray(x, float)
ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (3130,) + inhomogeneous part.

Traceback (most recent call last):
  File "/opt/cloudera/parcels/SPARK3-3.3.2.3.3.7190.2-1-1.p0.46867244/lib/spark3/python/lib/pyspark.zip/pyspark/sql/dataframe.py", line 606, in show
    print(self._jdf.showString(n, 20, vertical))
  File "/opt/cloudera/parcels/SPARK3-3.3.2.3.3.7190.2-1-1.p0.46867244/lib/spark3/python/lib/py4j-0.10.9.5-sr

In [59]:
# Inspect the structure of lausanne_geometry
print(broadcast_polygon)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 2) Filtering and Parsing

In [6]:
distinct_arr_prog_status = istdaten_df.select("arr_prog_status").distinct().collect()
distinct_arr_prog_status_list = [row.arr_prog_status for row in distinct_arr_prog_status]
print(distinct_arr_prog_status_list)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['GESCHAETZT', 'PROGNOSE', '', 'REAL', 'UNBEKANNT']

In [7]:
from pyspark.sql.functions import to_timestamp, col
format_date = "dd.MM.yyyy"
format_timetable = "dd.MM.yyyy HH:mm"
format_prognosis = "dd.MM.yyyy HH:mm:ss"

istdaten_df = istdaten_df.withColumn("date", to_timestamp(col("date"), format_date))\
                         .withColumn("arrival_time", to_timestamp(col("arrival_time"), format_timetable))\
                         .withColumn("arrival_prognosis", to_timestamp(col("arrival_prognosis"), format_prognosis))\
                         .withColumn("departure_time", to_timestamp(col("departure_time"), format_timetable))\
                         .withColumn("departure_prognosis", to_timestamp(col("departure_prognosis"), format_prognosis))

istdaten_df = istdaten_df.filter((col("arr_prog_status") != "") & (col("arr_prog_status") != "UNBEKANNT") & (col('cancelled') == 'false'))

istdaten_df.show(5)
istdaten_df.printSchema()
istdaten_df.count()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+--------------+-----------+-------+--------------+---------+-------------------+-------------------+---------------+-------------------+-------------------+---------------+
|               date|       trip_id|operator_id|stop_id|transportation|cancelled|       arrival_time|  arrival_prognosis|arr_prog_status|     departure_time|departure_prognosis|dep_prog_status|
+-------------------+--------------+-----------+-------+--------------+---------+-------------------+-------------------+---------------+-------------------+-------------------+---------------+
|2023-12-22 00:00:00|85:65:8031:001|      85:65|8506101|           Zug|    false|2023-12-22 09:36:00|2023-12-22 09:36:34|           REAL|2023-12-22 09:36:00|2023-12-22 09:36:53|           REAL|
|2023-12-22 00:00:00|85:65:8031:001|      85:65|8506102|           Zug|    false|2023-12-22 09:39:00|2023-12-22 09:39:08|           REAL|2023-12-22 09:39:00|2023-12-22 09:39:26|           REAL|
|2023-12-22 00:00:00|85:65:803

### 3) Istdaten Feature Engineering

In [8]:
from pyspark.sql.functions import col, to_date, year, month, dayofmonth, dayofweek, expr, when
from pyspark.sql.types import IntegerType

istdaten_df = istdaten_df.withColumn("delay_arrival", (col("arrival_prognosis").cast("long") - col("arrival_time").cast("long")))\
                         .withColumn("delay_departure", (col("departure_prognosis").cast("long") - col("departure_time").cast("long")))\
                         .withColumn("delay_at_station", (col("delay_departure") - col("delay_arrival")))\
                         .withColumn("year", year(col("date")))\
                         .withColumn("month", month(col("date")))\
                         .withColumn("day", dayofmonth(col("date")))\
                         .withColumn("day_of_week", (dayofweek(col("date")) + 5) % 7)\
                         .withColumn("is_weekday", (col("day_of_week") <= 4))\
                         .withColumn("is_weekend", (col("day_of_week") > 4))\
                         .withColumn("is_peak_time",
                                     when((col("is_weekday")) & 
                                          ((col("arrival_time").between(expr("make_timestamp(year(date), month(date), day(date), 6, 30, 0)"), 
                                                                        expr("make_timestamp(year(date), month(date), day(date), 8, 30, 0)"))) |
                                           (col("arrival_time").between(expr("make_timestamp(year(date), month(date), day(date), 16, 30, 0)"), 
                                                                        expr("make_timestamp(year(date), month(date), day(date), 18, 30, 0)")))), True)
                                     .otherwise(False))
istdaten_df.show(5)
istdaten_df.printSchema()
istdaten_df.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+--------------+-----------+-------+--------------+---------+-------------------+-------------------+---------------+-------------------+-------------------+---------------+-------------+---------------+----------------+----+-----+---+-----------+----------+----------+------------+
|               date|       trip_id|operator_id|stop_id|transportation|cancelled|       arrival_time|  arrival_prognosis|arr_prog_status|     departure_time|departure_prognosis|dep_prog_status|delay_arrival|delay_departure|delay_at_station|year|month|day|day_of_week|is_weekday|is_weekend|is_peak_time|
+-------------------+--------------+-----------+-------+--------------+---------+-------------------+-------------------+---------------+-------------------+-------------------+---------------+-------------+---------------+----------------+----+-----+---+-----------+----------+----------+------------+
|2023-12-22 00:00:00|85:65:8031:001|      85:65|8506101|           Zug|    false|2023-12-22

### 4) Additional Features

In [9]:
from pyspark.sql.types import DoubleType
stops_df = spark.read.csv('/data/sbb/csv/timetables/stops', header=True)\
                .select(['stop_id','stop_lat', 'stop_lon'])\
                .drop(*['year','month','day'])\
                .dropDuplicates(['stop_id'])\
                .withColumnRenamed("stop_id", "stops_stop_id")\
                .withColumn("stop_lat", col("stop_lat").cast(DoubleType()))\
                .withColumn("stop_lon", col("stop_lon").cast(DoubleType()))

istdaten_df = istdaten_df.join(
    stops_df,
    istdaten_df.stop_id == stops_df.stops_stop_id,
    "left"
).drop('stops_stop_id')

istdaten_df.show(5)
istdaten_df.printSchema()
istdaten_df.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+--------------------+-----------+-------+--------------+---------+-------------------+-------------------+---------------+-------------------+-------------------+---------------+-------------+---------------+----------------+----+-----+---+-----------+----------+----------+------------+----------------+----------------+
|               date|             trip_id|operator_id|stop_id|transportation|cancelled|       arrival_time|  arrival_prognosis|arr_prog_status|     departure_time|departure_prognosis|dep_prog_status|delay_arrival|delay_departure|delay_at_station|year|month|day|day_of_week|is_weekday|is_weekend|is_peak_time|        stop_lat|        stop_lon|
+-------------------+--------------------+-----------+-------+--------------+---------+-------------------+-------------------+---------------+-------------------+-------------------+---------------+-------------+---------------+----------------+----+-----+---+-----------+----------+----------+------------+------

In [10]:
from pyspark.sql.window import Window
from pyspark.sql.functions import udf, count, lag, col, to_timestamp, concat_ws, lit, lpad
from pyspark.sql.types import DoubleType
import math

# Haversine formula to calculate the distance between two points on the earth
def haversine(lat1, lon1, lat2, lon2):
    if lat1 is None or lon1 is None or lat2 is None or lon2 is None:
        return None
    R = 6371  # Radius of the Earth in kilometers
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat / 2) * math.sin(dlat / 2) + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon / 2) * math.sin(dlon / 2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c  # Distance in kilometers
    return distance

# Register the UDF
haversine_udf = udf(haversine, DoubleType())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
from pyspark.sql import Window
from pyspark.sql.functions import col, row_number, count

# Define the window spec for calculating trip_stop_seq with year, month, and day
trip_stop_seq_window = Window.partitionBy("trip_id", "year", "month", "day").orderBy("arrival_time")
istdaten_df = istdaten_df.withColumn("trip_stop_seq", row_number().over(trip_stop_seq_window))

# Define the window spec for calculating trip_num_stops with year, month, and day
trip_num_stops_window = Window.partitionBy("trip_id", "year", "month", "day")
istdaten_df = istdaten_df.withColumn("trip_num_stops", count("stop_id").over(trip_num_stops_window))

# Add columns for the previous stop_id and departure_time
window_spec_seq = Window.partitionBy("trip_id", "year", "month", "day").orderBy("trip_stop_seq")
istdaten_df = istdaten_df.withColumn("prev_stop_id", lag("stop_id", 1).over(window_spec_seq))\
                         .withColumn("prev_departure_time", lag("departure_time", 1).over(window_spec_seq))\
                         .withColumn("prev_stop_lat", lag("stop_lat", 1).over(window_spec_seq))\
                         .withColumn("prev_stop_lon", lag("stop_lon", 1).over(window_spec_seq))\
                         .withColumn("traveling_time_min", ((col("arrival_time").cast("long") - col("prev_departure_time").cast("long"))) / 60)\
                         .withColumn("traveling_distance_km", haversine_udf(col("prev_stop_lat"), col("prev_stop_lon"),col("stop_lat"), col("stop_lon")))

istdaten_df.show(5)
istdaten_df.printSchema()
istdaten_df.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-------------------+-----------+-------+--------------+---------+-------------------+-------------------+---------------+--------------+-------------------+---------------+-------------+---------------+----------------+----+-----+---+-----------+----------+----------+------------+----------------+----------------+-------------+--------------+------------+-------------------+-------------+-------------+------------------+---------------------+
|               date|            trip_id|operator_id|stop_id|transportation|cancelled|       arrival_time|  arrival_prognosis|arr_prog_status|departure_time|departure_prognosis|dep_prog_status|delay_arrival|delay_departure|delay_at_station|year|month|day|day_of_week|is_weekday|is_weekend|is_peak_time|        stop_lat|        stop_lon|trip_stop_seq|trip_num_stops|prev_stop_id|prev_departure_time|prev_stop_lat|prev_stop_lon|traveling_time_min|traveling_distance_km|
+-------------------+-------------------+-----------+-------+---

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

# Calculate the distance between each stop and each weather station
wstations_df = spark.read.csv('/data/wunderground/csv/stations', header=True)
distance_df = stops_df.crossJoin(wstations_df).withColumn(
    "distance",
    haversine_udf(col("stop_lat"),col("stop_lon"), col("lat_wgs84").cast(DoubleType()), col("lon_wgs84").cast(DoubleType()))
)

# Using window function to find the nearest station
windowSpec = Window.partitionBy("stops_stop_id").orderBy("distance")
nearest_stations = distance_df.withColumn("rank", row_number().over(windowSpec)) \
                              .filter(col("rank") == 1) \
                              .drop("rank")\
                              .select(['stops_stop_id', 'site', 'lat_wgs84', 'lon_wgs84', 'distance'])

# Show the results
nearest_stations.show(5)
nearest_stations.printSchema()
nearest_stations.count()

In [ ]:
weather_data_df = spark.read.orc('/data/share/weather_win3h_avg_df.orc')

weather_data_df.printSchema()
weather_data_df.show(5)

In [ ]:
from pyspark.sql.functions import unix_timestamp, to_timestamp, round, from_unixtime, hour
from pyspark.sql.functions import broadcast

weather_data_df = spark.read.orc('/data/share/weather_win3h_avg_df.orc')

# Merge weather data with station coordinates
weather_stations_df = weather_data_df.join(
    nearest_stations,
    weather_data_df.site == nearest_stations.site,
    "inner"
)

# Select necessary columns including coordinates in WGS84 which will help in geographical matching
weather_stations_df = weather_stations_df.select(
    "valid_time_gmt", "year", "month", "dayofmonth", "hour", 
    "temp", "avg_temp", "lat_wgs84", "lon_wgs84", "stops_stop_id"
).withColumn("weather_hour", (col("valid_time_gmt") / 3600).cast("integer") * 3600)

istdaten_df = istdaten_df.withColumn("arrival_unix", unix_timestamp(to_timestamp(col("arrival_time"))))\
                         .withColumn("arrival_unix", (col("arrival_unix") / 3600).cast("integer") * 3600)

istdaten_df = istdaten_df.join(
    weather_stations_df,
    (istdaten_df.stop_id == weather_stations_df.stops_stop_id) &
    (istdaten_df.arrival_unix == weather_stations_df.weather_hour),
    "left_outer"  # left_outer to keep transport data regardless of match
)

istdaten_df.show(5)

### Feature Selection

In [12]:
from pyspark.sql.functions import hour
features = istdaten_df.select(['stop_id','transportation','arrival_time','delay_arrival','year','month','day','day_of_week','is_weekend','is_peak_time',
                               'stop_lat','stop_lon','trip_stop_seq','trip_num_stops','traveling_time_min','traveling_distance_km'])\
                      .withColumn("arrival_hour", hour(col("arrival_time")))\

features.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------+-------------------+-------------+----+-----+---+-----------+----------+------------+----------------+----------------+-------------+--------------+------------------+---------------------+------------+
|stop_id|transportation|       arrival_time|delay_arrival|year|month|day|day_of_week|is_weekend|is_peak_time|        stop_lat|        stop_lon|trip_stop_seq|trip_num_stops|traveling_time_min|traveling_distance_km|arrival_hour|
+-------+--------------+-------------------+-------------+----+-----+---+-----------+----------+------------+----------------+----------------+-------------+--------------+------------------+---------------------+------------+
|8500090|           Zug|2023-01-05 17:23:00|          540|2023|    1|  5|          3|     false|        true|47.5673090682292|7.60691586067732|            1|             1|              null|                 null|          17|
|8500090|           Zug|2023-01-07 13:50:00|          240|2023|    1|  7|          5|      t

In [13]:
from pyspark.sql import functions as F

# Compute mean and std deviation for relevant columns per stop_id
statistics_df = features.groupBy('stop_id').agg(
    F.mean('delay_arrival').alias('mean_delay_arrival'),
    F.stddev('delay_arrival').alias('stddev_delay_arrival'),
    F.mean('traveling_time_min').alias('mean_traveling_time_min'),
    F.stddev('traveling_time_min').alias('stddev_traveling_time_min'),
    F.mean('traveling_distance_km').alias('mean_traveling_distance_km'),
    F.stddev('traveling_distance_km').alias('stddev_traveling_distance_km')
)

# Join the statistics back to the original dataframe
features_with_stats = features.join(statistics_df, on='stop_id', how='left')

# Show the result
features_with_stats.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------+-------------------+-------------+----+-----+---+-----------+----------+------------+----------------+----------------+-------------+--------------+------------------+---------------------+------------+------------------+--------------------+-----------------------+-------------------------+--------------------------+----------------------------+
|stop_id|transportation|       arrival_time|delay_arrival|year|month|day|day_of_week|is_weekend|is_peak_time|        stop_lat|        stop_lon|trip_stop_seq|trip_num_stops|traveling_time_min|traveling_distance_km|arrival_hour|mean_delay_arrival|stddev_delay_arrival|mean_traveling_time_min|stddev_traveling_time_min|mean_traveling_distance_km|stddev_traveling_distance_km|
+-------+--------------+-------------------+-------------+----+-----+---+-----------+----------+------------+----------------+----------------+-------------+--------------+------------------+---------------------+------------+------------------+---------

In [16]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer

# Select features and target variable
feature_columns = ['transportation', 'year', 'month', 'day', 'day_of_week', 'is_weekend', 'is_peak_time', 
                   'stop_lat', 'stop_lon', 'trip_stop_seq', 'trip_num_stops', 'traveling_time_min', 
                   'traveling_distance_km', 'arrival_hour', 'mean_delay_arrival', 'stddev_delay_arrival', 
                   'mean_traveling_time_min', 'stddev_traveling_time_min', 'mean_traveling_distance_km', 
                   'stddev_traveling_distance_km']
target_column = 'delay_arrival'


# Convert categorical features to numeric
# Index the 'transportation' column
indexer = StringIndexer(inputCol="transportation", outputCol="transportation_indexed")

# Assemble features into a single vector
assembler = VectorAssembler(inputCols=['transportation_indexed'] + feature_columns[1:], outputCol="features")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
# Filter out rows with null values
features_with_stats = features_with_stats.na.drop()

# Sample the data for initial model training and testing --> no sampling leads to crash (out of memory)
# Can be solved in two ways:
# 1. Providing additional resources -> expanding RAM and number of cores
# 2. Data processing -> filtering on select stops to remove less relevant rows
sampled_data = features_with_stats.sample(withReplacement=False, fraction=0.1, seed=42)

# Define the Random Forest Regressor
rf = RandomForestRegressor(featuresCol="features", labelCol=target_column)

# Create a pipeline
pipeline = Pipeline(stages=[indexer, assembler, rf])

# Split the data into training and testing sets
train_data, test_data = sampled_data.randomSplit([0.8, 0.2], seed=42)

# Train the model
model = pipeline.fit(train_data)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
An error occurred while calling o618.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 83 in stage 254.0 failed 4 times, most recent failure: Lost task 83.3 in stage 254.0 (TID 13013) (iccluster065.iccluster.epfl.ch executor 639): ExecutorLostFailure (executor 639 exited caused by one of the running tasks) Reason: Container killed by YARN for exceeding physical memory limits. 5.1 GB of 5 GB physical memory used. Consider boosting spark.executor.memoryOverhead.
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2672)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2608)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2607)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collectio

In [ ]:
# Make predictions
predictions = model.transform(test_data)

# Evaluate the model
evaluator = RegressionEvaluator(labelCol=target_column, predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print(f"Root Mean Squared Error (RMSE): {rmse}")

# Show some predictions
predictions.select("stop_id", "arrival_time", target_column, "prediction").show(5)